In [1]:
import pandas as pd
import polyline



import numpy as np

from numpy.linalg import norm

from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer 
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

 
#Оптимизация буста    
import xgboost as xgb
from sklearn import preprocessing
# from sklearn.cross_validation import train_test_split
# from sklearn.cross_validation import *

from sklearn.model_selection import GridSearchCV
#from sklearn.grid_search import GridSearchCV




In [2]:
# test.csv – выборка для предсказания
# test_additional.csv – выборка для предсказания + дополнительные поля (e.g координаты проставки статусов и др.)
# train.csv – выборка для обучения
# validation.csv – выбора для валидации
# Описание полей
# Id

# main_id_locality – Идентификатор города
# ETA – Ожидаемое время поездки
# RTA – Реальное время поездки / Требуется предсказать
# OrderedDate – Время заказа
# latitude – Широта старта заказа
# del_latitude – Широта конца заказа
# longitude – Долгота старта заказа
# del_longitude – Долгота конца заказа
# EDA – Ожидаемая дистанция заказа


# RDA – Фактическая дистанция заказа / Нет на момент создания заказа
# ReadyForCollection – Время события "Подъехал" / Нет на момент создания заказа
# ClientCollected – Время события "В пути" / Нет на момент создания заказа
# GoodArrived – Время события "Прибыл" / Нет на момент создания заказа
# ready_latitude – Широта события "Подъехал" / Нет на момент создания заказа
# ready_longitude – Долгота события "Подъехал" / Нет на момент создания заказа
# onway_latitude – Широта события "В пути" / Нет на момент создания заказа
# onway_longitude – Долгота события "В пути" / Нет на момент создания заказа
# arrived_latitude – Широта события "Прибыл" / Нет на момент создания заказа
# arrived_longitude – Долгота события "Прибыл" / Нет на момент создания заказа


# center_latitude – Широта центра города
# center_longitude – Долгота центра города
# route – расчетный трек

In [3]:
p = np.array([4, 6])
q = np.array([1, 2])
r = np.array([0, -1])

def t(p, q, r):
    x = p-q
    return np.dot(r-q, x)/np.dot(x, x)

def d(p, q, r):
    return np.linalg.norm(t(p, q, r)*(p-q)+q-r)
print(p)
print(d(p, q, r))

[4 6]
0.9999999999999999


In [4]:
# def t(p, q, r):
#     x = p-q
#     return np.dot(r-q, x)/np.dot(x, x)

# def d(p, q, r):
#     return np.linalg.norm(t(p, q, r)*(p-q)+q-r)

# f = pd.read_csv('train.csv')
# f.route.fillna(0, inplace=True)

# x=f['longitude']
# y=f['latitude']
# #p = np.hstack((x, y))
# P=np.stack((x, y), axis=-1)

# x=f['del_longitude']
# y=f['del_latitude']
# Q=np.stack((x, y), axis=-1)

# x=f['center_longitude']
# y=f['center_latitude']
# R=np.stack((x, y), axis=-1)

# dist=[]
# for i in range(len(R)):
#     p = P[i]
#     q = Q[i]
#     r = R[i]
#     dist.append(d(p, q, r))
    
# f['distance']=dist

In [5]:
# f['distance'].head()

# Список фичей 

In [6]:
# Близость к центру
# (А,Б, середина)

# Праздник

# Диапазон цены

# Расстояние между точками   (Мантхетен и Евклид)

# Сложность маршрута  

# GridSearchCV параметров модели

# Комбинация нескольких алгоритмов. Стекинг

# Комбинация:            lightgbm, xgboost, catboost

# Возможно погода.

# Выходной или нет

# Направление от/до центра

# Величина шума

# Погрешность величины 
 
# Временное расстояние до ближайшего праздника

# Категориальные и числовые переменные

# N-way-split как в решающих деревьях

# Переобучение

# Валидация

# Разбить рандомно фичи на группы и скормить алгоритму (возможно уже)

# CatBoosting 

In [7]:
# f = pd.read_csv('train.csv')
# f.route.fillna(0, inplace=True)


# x=f['longitude']
# y=f['latitude']
# #p = np.hstack((x, y))
# p=np.stack((x, y), axis=-1)

# x=f['del_longitude']
# y=f['del_latitude']
# q=np.stack((x, y), axis=-1)

# x=f['center_longitude']
# y=f['center_latitude']
# r=np.stack((x, y), axis=-1)


# print(p)

In [8]:
f = pd.read_csv('train.csv')
er=abs(f['RTA']-f['ETA']).mean()
er

209.33793517837177

In [9]:
# f = pd.read_csv('train.csv')
# f.route.fillna(0, inplace=True)
# r=polyline.decode(f['route'].values[:5])
# r=len(r)

# Дизай Фичей


In [10]:
f = pd.read_csv('train.csv')


#er=abs(f['RTA']-f['ETA']).mean()
#Опираться нельзя
f=f.drop(['RDA', 'ReadyForCollection','ClientCollected','GoodArrived','ready_latitude', 'ready_longitude','onway_latitude','onway_longitude','arrived_latitude', 'arrived_longitude'], axis=1)
#Опираться непонтяно как
f=f.drop(['track'], axis=1)
#Опираться бессмысленно
f=f.drop(['Id'], axis=1)
f.route.fillna(0, inplace=True)

#'center_longitude', 'center_latitude'

#Разбиваем дату на фичи: 
f.OrderedDate=f.OrderedDate.astype('datetime64[ns]')    # Разпознаем дату

f['minute']=f.OrderedDate.dt.minute
f['hour']=f.OrderedDate.dt.hour
f['week_day']=f.OrderedDate.dt.weekday
f['month_day']=f.OrderedDate.dt.day
f['month']=f.OrderedDate.dt.month

f=f.drop(['OrderedDate'], axis=1)    #Дропаем дату


# #берем расстояния
f['A_B_evklid']=(f['latitude']-f['del_latitude'])**2+(f['longitude']-f['del_longitude'])**2
f['A_B_manhet']=abs(f['latitude']-f['del_latitude'])+abs(f['longitude']-f['del_longitude'])

f['points']=polyline.decode(train.iloc[2]['route']).sum()


def t(p, q, r):
    x = p-q
    return np.dot(r-q, x)/np.dot(x, x)

def d(p, q, r):
    return np.linalg.norm(t(p, q, r)*(p-q)+q-r)


x=f['longitude']
y=f['latitude']
#p = np.hstack((x, y))
P=np.stack((x, y), axis=-1)

x=f['del_longitude']
y=f['del_latitude']
Q=np.stack((x, y), axis=-1)

x=f['center_longitude']
y=f['center_latitude']
R=np.stack((x, y), axis=-1)

dist=[]
for i in range(len(R)):
    p = P[i]
    q = Q[i]
    r = R[i]
    dist.append(d(p, q, r))
    
f['distance']=dist


In [11]:
X=f.drop(['RTA'], axis=1)   #Данный для предсказания
y=f['RTA']                  #Предсказуемая величина

#X.head()

In [12]:
f.sample(6)

,main_id_locality,ETA,RTA,latitude,del_latitude,longitude,del_longitude,EDA,center_latitude,center_longitude,minute,hour,week_day,month_day,month,A_B_evklid,A_B_manhet,distance
299259,1078,1852.0,1309.0,55.795059,55.766602,49.191002,49.102482,9.0,55.794388,49.111531,11,10,2,19,2,0.008646,0.116978,0.023683
136156,1078,989.0,759.0,55.778259,55.839119,49.131519,49.135540,9.0,55.794388,49.111531,14,22,3,23,1,0.003720,0.064880,0.021008
632440,22430,552.0,332.0,51.542950,51.526440,46.011471,46.031940,3.0,51.530376,45.953026,25,23,2,4,3,0.000692,0.036980,0.046479
729766,22438,442.0,257.0,57.988258,57.976151,56.218689,56.208370,2.0,58.010321,56.234178,29,20,4,13,3,0.000253,0.022427,0.002522
242222,1078,1358.0,956.0,55.689751,55.751339,49.146320,49.174309,13.0,55.794388,49.111531,44,16,6,26,1,0.004576,0.089577,0.074963
316192,22390,737.0,527.0,53.265572,53.226669,50.208210,50.191662,6.0,53.183330,50.116670,25,22,2,12,2,0.001787,0.055450,0.052043


In [13]:
# from sklearn.impute import SimpleImputer 
# imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# instead of

# from sklearn.preprocessing import Imputer
# imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)

In [14]:
#'DataFrame' object has no attribute 'as_matrix'

# coords = df.as_matrix(columns=['Latitude','Longitude'])
# instead of

# coords = df[["Latitude", "Longitude"]].values

# Модель XGBoosting

In [15]:
# ### n_jobs
# On larger datasets where runtime is a consideration, you can use parallelism to build your models faster.  It's common to set the parameter **n_jobs** equal to the number of cores on your machine.  On smaller datasets, this won't help. 

# The resulting model won't be any better, so micro-optimizing for fitting time is typically nothing but a distraction. But, it's useful in large datasets where you would otherwise spend a long time waiting during the `fit` command.

# XGBoost has a multitude of other parameters, but these will go a very long way in helping you fine-tune your XGBoost model for optimal performance.

# # Conclusion
# XGBoost is currently the dominant algorithm for building accurate models on conventional data (also called tabular or strutured data).  Go apply it to improve your models!

# # Your Turn
# Convert yuor model to use XGBoost.

# Use early stopping to find a good value for n_estimators.  Then re-estimate the model with all of your training data, and that value of n_estimators.

# Once you've done this, return to **[Learning Machine Learning](https://www.kaggle.com/dansbecker/learn-machine-learning)**, to keep improving..  




data = f #pd.read_csv('../input/train.csv')
data.dropna(axis=0, subset=['RTA'], inplace=True)

y = data.RTA
X = data.drop(['RTA'], axis=1).select_dtypes(exclude=['object'])

#train_X, test_X, train_y, test_y = train_test_split(X.as_matrix(), y.as_matrix(), test_size=0.25)
train_X, test_X, train_y, test_y = train_test_split(X.values, y.values, test_size=0.25)


#my_imputer = Imputer()
my_imputer = SimpleImputer()


train_X = my_imputer.fit_transform(train_X)
test_X = my_imputer.transform(test_X)



# my_model = XGBRegressor()
# #  add verbose=False to avoid printing out updates with each cycle
# my_model.fit(train_X, train_y, verbose=False)
# # make predictions
# predictions = my_model.predict(test_X)
# print("Mean Absolute Error : " + str(mean_absolute_error(predictions, test_y)))
# # Mean Absolute Error : 18017.7953339 
# # better than Mean Absolute Error with One-Hot Encoding (18052)
# # fitting with early_stopping:


# my_model = XGBRegressor(n_estimators=1000)
# my_model.fit(train_X, train_y, early_stopping_rounds=5, 
#              eval_set=[(test_X, test_y)], verbose=False)
# predictions = my_model.predict(test_X)
# print("Early_Stopping Mean Absolute Error : " + str(mean_absolute_error(predictions, test_y)))
# # Early_Stopping Mean Absolute Error : 18241.4572346

from sklearn.model_selection import StratifiedKFold

my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
#my_model = XGBRegressor()
#my_model

# xgb_model = xgb.XGBClassifier()
# #brute force scan for all parameters, here are the tricks
# #usually max_depth is 6,7,8
# #learning rate is around 0.05, but small changes may make big diff
# #tuning min_child_weight subsample colsample_bytree can have 
# #much fun of fighting against overfit 
# #n_estimators is how many round of boosting
# #finally, ensemble xgboost with multiple seeds may reduce variance

# parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
 
#               'learning_rate': [0.05], #so called `eta` value
#               'max_depth': [5,6,10],
#               'min_child_weight': [11],
#               'silent': [1],
#               'subsample': [0.8],
#               'colsample_bytree': [0.7],
#               'n_estimators': [1001],}
 
 


# my_model = GridSearchCV(my_model, parameters, n_jobs=5, 
#                    cv=5,    #StratifiedKFold(y, n_folds=5, shuffle=True)
#                    scoring='roc_auc',
#                    verbose=2, refit=True)

# clf.fit(train[features], train["QuoteConversion_Flag"])

In [16]:
#best model be here

In [17]:
# # including a learning rate
# my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
# my_model

In [18]:
my_model = XGBRegressor(n_estimators=1100, learning_rate=0.05)
my_model.fit(train_X, train_y, early_stopping_rounds=5, 
             eval_set=[(test_X, test_y)], verbose=False)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=1100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
             validate_parameters=False, verbosity=None)

In [19]:
predictions1 = my_model.predict(test_X)

In [20]:
predictions1


array([738.83856, 314.28247, 723.5157 , ..., 339.86362, 494.87024,
       851.2008 ], dtype=float32)

In [36]:
print(predictions1)

[738.83856 314.28247 723.5157  ... 339.86362 494.87024 851.2008 ]


In [22]:
err = abs(y-predictions1).mean()

ValueError: operands could not be broadcast together with shapes (837128,) (209282,) 

In [23]:
print("ES and LR Mean Absolute Error : " + str(mean_absolute_error(predictions, test_y)))
# ES and LR Mean Absolute Error : 18138.6426156 
print(my_model)

NameError: name 'predictions' is not defined

In [24]:
import numpy as np
from sklearn.linear_model import LinearRegression
reg = LinearRegression(normalize=True).fit(train_X, train_y)
predictions2 = reg.predict(test_X)
predictions2


array([774.19514184, 314.70217775, 728.61121104, ..., 391.15534005,
       449.74921641, 857.77572385])

# Стекинг Моделей

In [25]:
predictions=(predictions1+predictions2)/2

print("ES and LR Mean Absolute Error 2: " + str(mean_absolute_error(predictions, test_y)))
# ES and LR Mean Absolute Error : 18138.6426156 
print(my_model)

ES and LR Mean Absolute Error 2: 111.31689265955045
XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=1100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
             validate_parameters=False, verbosity=None)


# Итоговая ошибка

In [26]:
print("ES and LR Mean Absolute Error final: " + str(mean_absolute_error(predictions, test_y)))
# ES and LR Mean Absolute Error : 18138.6426156 
print(my_model)

ES and LR Mean Absolute Error final: 111.31689265955045
XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=1100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
             validate_parameters=False, verbosity=None)


# Оптимизация фистинга... ой, - бустинга!


In [27]:
# # my_model = XGBRegressor(n_estimators=1000)
# # my_model.fit(train_X, train_y, early_stopping_rounds=5, 
# #              eval_set=[(test_X, test_y)], verbose=False)
# # predictions = my_model.predict(test_X)
# # print("Early_Stopping Mean Absolute Error : " + str(mean_absolute_error(predictions, test_y)))
# # # Early_Stopping Mean Absolute Error : 18241.4572346

# from sklearn.model_selection import StratifiedKFold

# my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
# #my_model = XGBRegressor()
# #my_model

# # xgb_model = xgb.XGBClassifier()
# # #brute force scan for all parameters, here are the tricks
# # #usually max_depth is 6,7,8
# # #learning rate is around 0.05, but small changes may make big diff
# # #tuning min_child_weight subsample colsample_bytree can have 
# # #much fun of fighting against overfit 
# # #n_estimators is how many round of boosting
# # #finally, ensemble xgboost with multiple seeds may reduce variance

# # parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
 
# #               'learning_rate': [0.05], #so called `eta` value
# #               'max_depth': [5,6,10],
# #               'min_child_weight': [11],
# #               'silent': [1],
# #               'subsample': [0.8],
# #               'colsample_bytree': [0.7],
# #               'n_estimators': [1001],}

# Предсказание. Финал

In [28]:
predictions

array([756.51685192, 314.49232423, 726.06344854, ..., 365.5094785 ,
       472.30972783, 854.48826476])

In [29]:
g=pd.read_csv('test.csv')
g=g.drop(['Id'], axis=1)
g.route.fillna(0, inplace=True)
g.OrderedDate=g.OrderedDate.astype('datetime64[ns]')    # Разпознаем дату
g['minute']=g.OrderedDate.dt.minute
g['hour']=g.OrderedDate.dt.hour
g['week_day']=g.OrderedDate.dt.weekday
g['month_day']=g.OrderedDate.dt.day
g['month']=g.OrderedDate.dt.month
g=g.drop(['OrderedDate'], axis=1)    #Дропаем дату

g['A_B_evklid']=(g['latitude']-g['del_latitude'])**2+(g['longitude']-g['del_longitude'])**2
g['A_B_manhet']=abs(g['latitude']-g['del_latitude'])+abs(g['longitude']-g['del_longitude'])

g['points']=polyline.decode(train.iloc[2]['route']).sum()
def t(p, q, r):
    x = p-q
    return np.dot(r-q, x)/np.dot(x, x)

def d(p, q, r):
    return np.linalg.norm(t(p, q, r)*(p-q)+q-r)


x=g['longitude']
y=g['latitude']
#p = np.hstack((x, y))
P=np.stack((x, y), axis=-1)

x=g['del_longitude']
y=g['del_latitude']
Q=np.stack((x, y), axis=-1)

x=g['center_longitude']
y=g['center_latitude']
R=np.stack((x, y), axis=-1)

dist=[]
for i in range(len(R)):
    p = P[i]
    q = Q[i]
    r = R[i]
    dist.append(d(p, q, r))
    
g['distance']=dist


test_x_for_submit=g
test_x_for_submit.head()

AttributeError: 'DataFrame' object has no attribute 'route'

In [ ]:
predictions = my_model.predict(test_x_for_submit.values)
predictions

In [ ]:
predictions2 = reg.predict(test_x_for_submit.values)
#predictions=(predictions2+predictions)/2

In [30]:
ID=pd.read_csv('test.csv')
ID.head()

,Id,main_id_locality,ETA,OrderedDate,latitude,del_latitude,longitude,del_longitude,EDA,center_latitude,center_longitude
0,0,1078,1212.0,2020-03-28 00:00:05,55.751270,55.836491,49.183281,49.140308,13.0,55.794388,49.111531
1,1,1078,968.0,2020-03-28 00:00:29,55.829720,55.819080,49.116032,49.183521,8.0,55.794388,49.111531
2,2,1078,492.0,2020-03-28 00:00:31,55.785660,55.783699,49.173920,49.195599,3.0,55.794388,49.111531
3,3,1078,953.0,2020-03-28 00:00:42,55.792568,55.824730,49.172691,49.084911,9.0,55.794388,49.111531
4,4,1078,1746.0,2020-03-28 00:00:47,55.829369,55.757278,49.092682,49.236542,16.0,55.794388,49.111531


In [31]:
predictions

array([756.51685192, 314.49232423, 726.06344854, ..., 365.5094785 ,
       472.30972783, 854.48826476])

In [32]:

numpy_data = np.array([[1], [3]])
df = pd.DataFrame(data=predictions, columns=["Prediction"])

print(df.head())
ID=ID.join(df)


ID=ID[['Id','Prediction']]
 
ID.to_csv('submit.csv',index=False)
ID_open=pd.read_csv('submit.csv')
ID_open.head()

   Prediction
0  756.516852
1  314.492324
2  726.063449
3  510.989450
4  494.361989


,Id,Prediction
0,0,756.516852
1,1,314.492324
2,2,726.063449
3,3,510.989450
4,4,494.361989


In [33]:
ID.to_csv('submit.csv',index=False)
ID_open=pd.read_csv('submit.csv')
ID_open.head()

,Id,Prediction
0,0,756.516852
1,1,314.492324
2,2,726.063449
3,3,510.989450
4,4,494.361989


In [34]:
# import matplotlib.pyplot as plt
# from scipy import interpolate
# import numpy as np

# y= [10.5,10,12]
# x= np.linspace(1, len(y), len(y), endpoint=True)
# dist = np.linalg.norm(y-x)

# print(dist)

# fig, ax = plt.subplots(figsize=(8, 6))
# ax.plot(x, y, color='red')

In [35]:
#=============================================================
# евклидово расстояние
#=============================================================
def euclidean_distance(v1, v2):
    return sum((x - y) ** 2 for x, y in zip(v1, v2)) ** 0.5

#=============================================================
# манхэттенское расстояние
#=============================================================
def manhattan_distance(v1, v2):
    return sum((x - y) for (x, y) in zip(v1, v2))



# Еще фичи

In [ ]:
# import numpy as np
# from numpy.linalg import norm

# def get_edges(fig):
#     return [[fig[i-1], fig[i]] for i in range(1, len(fig))] \
#            + [[fig[-1], fig[0]]]

# def dist_edge_to_point(l1, l2, p):
#     l1 = np.array(l1)
#     l2 = np.array(l2)
#     p = np.array(p)    
#     return norm(np.cross(l2-l1, l1-p))/norm(l2-l1)

# def min_dist_to_edge(p, nodes):
#     edges = get_edges(nodes)
#     data = [(dist_edge_to_point(l1, l2, p), l1, l2)
#             for l1,l2 in edges]
#     return min(data)

# figures = [
#     [(132, 0), (565, 234), (543, 103), (320, 129)],
#     [(342, 56), (159, 361), (623, 121)]
# ]

# figures = [[list(p) for p in fig] for fig in figures]

# p = (100, 200)

# min_dists = [min_dist_to_edge(p, nodes) for nodes in figures]
# min_dists